In [52]:
import os
import pandas as pd
import numpy as np

In [102]:
faq = pd.read_csv('./data/QuestionSet.csv')

In [103]:
faq.shape

(70, 4)

In [104]:
faq.head()

,family,question_main,intent,QuestionWords
0,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card
1,Applying and account information,Can I add an authorized user to my card?,Authuser,add auth authorized user card
2,Applying and account information,How can I find out the status of my credit car...,CardAppStatus,find update status credit card application
3,Applying and account information,When will I receive my new credit card?,NewCCReceive,when timing receive new card
4,Applying and account information,How do you apply my payments?,ApplyPayments,apply payments how principal interest


In [105]:
#Creates a bigger chat corpus from the question words
from itertools import permutations

newlist_df=[]
for index, row in faq.iterrows():
    sent = str(row['QuestionWords'])
    #sent = str(row['question_main'])
    row['q_perm'] = [' '.join(words) for words in permutations(sent.split(), 5)]
    #row['q_perm'] = [' '.join(words) for words in permutations(sent.split()) if random.uniform(0,1)>0.2] #Trying to source from questions
    newlist_df.append(row)

In [106]:
#Create a dataframe from the above list with the list having permutations of the different questions
new_df=pd.DataFrame(newlist_df)

In [107]:
new_df.shape

(70, 5)

In [108]:
#Creating dataframe with different row for each question
a = new_df.q_perm.apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame('q_perm')
faq_df = new_df.drop('q_perm', axis=1).join(a).reset_index(drop=True)

In [109]:
faq_df.shape

(36615, 5)

In [110]:
faq_df.head(5)

,family,question_main,intent,QuestionWords,q_perm
0,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply information need
1,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply information card
2,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply need information
3,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply need card
4,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply card information


In [111]:
## Updating with intent for greetings, thanks and bye
faq_df['q_perm'] = np.where(faq_df['q_perm'].isnull(),faq_df['question_main'],faq_df['q_perm'])


In [112]:
faq_df.head()

,family,question_main,intent,QuestionWords,q_perm
0,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply information need
1,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply information card
2,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply need information
3,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply need card
4,Applying and account information,What information does Banco Uno® require when ...,Cardapply,what how apply information need card,what how apply card information


In [113]:
faq_df[faq_df['intent']=='greeting']

,family,question_main,intent,QuestionWords,q_perm
36600,Greeting,Hi,greeting,Hi,Hi
36601,Greeting,How are you?,greeting,How are you?,How are you?
36602,Greeting,Hello,greeting,Hello,Hello
36603,Greeting,Hola,greeting,Hola,Hola
36604,Greeting,Hey,greeting,Hey,Hey


In [114]:
#Creating global dataset of recent Kiva loans
faq_df.to_csv('./data/ChatCorpus.csv', index=False)